In [1]:
import numpy as np
import pandas as pd

In [2]:
from gensim.models import Word2Vec

C:\Users\k\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Bidirectional, Dense, Dropout, Activation, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

C:\Users\k\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
max_len = 500
embedding_size = 100
dropout=0.5
epochs = 10
validation_split = 0.1
batch_size = 128

In [5]:
tokenizer = Tokenizer()

In [6]:
train = pd.read_pickle('train.pkl')

In [7]:
tokenizer.fit_on_texts(train.Content)

In [8]:
vocab_size = len(tokenizer.word_index)

In [9]:
word_vec = Word2Vec.load('./word2vec')

In [10]:
vocab_size

878856

In [11]:
train_seq = tokenizer.texts_to_sequences(train.Content)

In [12]:
X_train = pad_sequences(train_seq, maxlen=max_len, truncating='post', padding='post')

In [13]:
y_train = pd.get_dummies(train.Lable).values

In [14]:
embedding_matrix = np.zeros((vocab_size, embedding_size), dtype=np.float32)

In [15]:
unknown_count = 0
unknown_freq = {}
for word, index in tokenizer.word_index.items():
    if index >= vocab_size: 
            continue
    try:
        embedding_matrix[index, :] = word_vec.wv[word]
    except KeyError:
        unknown_freq[word] = tokenizer.word_counts[word]
        unknown_count += 1

In [16]:
def computation_graph():
    model  = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], name='Embedding_Layer'))
    model.add(Bidirectional(LSTM(units=100)))
#     model.add(Dense(units=hyperparam['filters'], name='Dense_'+str(hyperparam['filters'])))
    model.add(Dropout(rate=dropout, name = 'Dropout_' + str(dropout)))
#     model.add(Activation(hyperparam['dense_activation'], name='Activation_'+str(hyperparam['dense_activation'])))
    model.add(Dense(units=y_train.shape[1], activation='softmax', name='Dense_'+str(y_train.shape[1])+'_Softmax'))
    return model

In [17]:
model = computation_graph()

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, None, 100)         87885600  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
Dropout_0.5 (Dropout)        (None, 200)               0         
_________________________________________________________________
Dense_14_Softmax (Dense)     (None, 14)                2814      
Total params: 88,049,214
Trainable params: 88,049,214
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
history = model.fit(x=X_train, y=y_train,
                          validation_split = validation_split,
                          epochs=epochs,
                          batch_size=batch_size, 
                          shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 566s 13ms/step - loss: 1.1764 - acc: 0.6340 - val_loss: 0.7851 - val_acc: 0.7598
Epoch 2/10
45000/45000 [==============================] - 559s 12ms/step - loss: 0.9249 - acc: 0.7173 - val_loss: 0.8413 - val_acc: 0.7610
Epoch 3/10
45000/45000 [==============================] - 519s 12ms/step - loss: 0.7333 - acc: 0.7890 - val_loss: 0.6841 - val_acc: 0.7954
Epoch 4/10
45000/45000 [==============================] - 516s 11ms/step - loss: 0.5941 - acc: 0.8278 - val_loss: 0.6429 - val_acc: 0.8124
Epoch 5/10
45000/45000 [==============================] - 527s 12ms/step - loss: 0.4815 - acc: 0.8643 - val_loss: 0.6093 - val_acc: 0.8198
Epoch 6/10
45000/45000 [==============================] - 518s 12ms/step - loss: 0.3912 - acc: 0.8910 - val_loss: 0.6143 - val_acc: 0.8190
Epoch 7/10
45000/45000 [==============================] - 520s 12ms/step - loss: 0.3112 - acc: 0.9153 - val_loss: 0.6